# Language APIs

## Natural Language AI

In [56]:
from google.cloud import language

def analyze_text_sentiment(text: str):
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    results = client.analyze_sentiment(document=document)
    show_text_sentiment(results)

def show_text_sentiment(response: language.AnalyzeSentimentResponse):
    import pandas as pd

    columns = ["score", "sentence"]
    data = [(s.sentiment.score, s.text.content) for s in response.sentences]
    df_sentence = pd.DataFrame(columns=columns, data=data)

    sentiment = response.document_sentiment
    columns = ["score", "magnitude", "language"]
    data = [(sentiment.score, sentiment.magnitude, response.language)]
    df_document = pd.DataFrame(columns=columns, data=data)

    format_args = {
        "index": False, 
        "tablefmt": "pipe", 
        "floatfmt": "+.1f",
    }
    
    print(f"At sentence level:\n{df_sentence.to_markdown(**format_args)}")
    print()
    print(f"At document level:\n{df_document.to_markdown(**format_args)}\n\n")

In [66]:
text_list = [
"""
Python is a very readable language, which makes it easy to understand and maintain code.
It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks.
One disadvantage is its speed: it's not as fast as some other programming languages.
""",
"""
Python is a very readable language, which makes it easy to understand and maintain code.
It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks.
One disadvantage is its speed: it's not as fast as some other programming languages.
""" ]

# Send a request to the API
for index, text in enumerate(text_list):
    print(f"Text {index}")
    analyze_text_sentiment(text)
     print("
           \n\n")

IndentationError: unexpected indent (71199838.py, line 17)

## Text-to-speech

```
pip install google-cloud-texttospeech

```


In [31]:
import google.cloud.texttospeech as tts


def list_voices(language_code=None):
    client = tts.TextToSpeechClient()
    response = client.list_voices(language_code=language_code)
    voices = sorted(response.voices, key=lambda voice: voice.name)

    print(f" Voices: {len(voices)} ".center(60, "-"))
    for voice in voices:
        languages = ", ".join(voice.language_codes)
        name = voice.name
        gender = tts.SsmlVoiceGender(voice.ssml_gender).name
        rate = voice.natural_sample_rate_hertz
        print(f"{languages:<8} | {name:<24} | {gender:<8} | {rate:,} Hz")

In [32]:
list_voices("en")

------------------------ Voices: 89 ------------------------
en-AU    | en-AU-Neural2-A          | FEMALE   | 24,000 Hz
en-AU    | en-AU-Neural2-B          | MALE     | 24,000 Hz
en-AU    | en-AU-Neural2-C          | FEMALE   | 24,000 Hz
en-AU    | en-AU-Neural2-D          | MALE     | 24,000 Hz
en-AU    | en-AU-News-E             | FEMALE   | 24,000 Hz
en-AU    | en-AU-News-F             | FEMALE   | 24,000 Hz
en-AU    | en-AU-News-G             | MALE     | 24,000 Hz
en-AU    | en-AU-Polyglot-1         | MALE     | 24,000 Hz
en-AU    | en-AU-Standard-A         | FEMALE   | 24,000 Hz
en-AU    | en-AU-Standard-B         | MALE     | 24,000 Hz
en-AU    | en-AU-Standard-C         | FEMALE   | 24,000 Hz
en-AU    | en-AU-Standard-D         | MALE     | 24,000 Hz
en-AU    | en-AU-Wavenet-A          | FEMALE   | 24,000 Hz
en-AU    | en-AU-Wavenet-B          | MALE     | 24,000 Hz
en-AU    | en-AU-Wavenet-C          | FEMALE   | 24,000 Hz
en-AU    | en-AU-Wavenet-D          | MALE     | 24,00

In [8]:
import google.cloud.texttospeech as tts


def text_to_wav(voice_name: str, text: str):
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name
    )
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

    client = tts.TextToSpeechClient()
    response = client.synthesize_speech(
        input=text_input,
        voice=voice_params,
        audio_config=audio_config,
    )

    filename = f"{voice_name}.wav"
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f'Generated speech saved to "{filename}"')

In [9]:
voicename = "en-GB-Neural2-A"
text_to_wav(voicename, "IS this a good API?")
import IPython
IPython.display.Audio(voicename + ".wav")

Generated speech saved to "en-GB-Neural2-A.wav"


# Image and video API

## Vision AI

```
pip install google-cloud-vision
```

In [11]:
from google.cloud import vision

def analyze_image_from_uri(
    image_uri: str,
    feature_types: Sequence,
) -> vision.AnnotateImageResponse:
    client = vision.ImageAnnotatorClient()

    image = vision.Image()
    image.source.image_uri = image_uri
    features = [vision.Feature(type_=feature_type) for feature_type in feature_types]
    request = vision.AnnotateImageRequest(image=image, features=features)

    response = client.annotate_image(request=request)

    return response

def print_objects(response: vision.AnnotateImageResponse):
    print("=" * 80)
    for obj in response.localized_object_annotations:
        nvertices = obj.bounding_poly.normalized_vertices
        print(
            f"{obj.score:4.0%}",
            f"{obj.name:15}",
            f"{obj.mid:10}",
            ",".join(f"({v.x:.1f},{v.y:.1f})" for v in nvertices),
            sep=" | ",
        )

In [12]:
image_uri = "gs://cloud-samples-data/vision/label/setagaya.jpeg"
features = [vision.Feature.Type.OBJECT_LOCALIZATION]

response = analyze_image_from_uri(image_uri, features)
print_objects(response)

 93% | Bicycle         | /m/0199g   | (0.6,0.6),(0.8,0.6),(0.8,0.9),(0.6,0.9)
 92% | Bicycle wheel   | /m/01bqk0  | (0.6,0.7),(0.7,0.7),(0.7,0.9),(0.6,0.9)
 91% | Tire            | /m/0h9mv   | (0.7,0.7),(0.8,0.7),(0.8,1.0),(0.7,1.0)
 75% | Bicycle         | /m/0199g   | (0.3,0.6),(0.4,0.6),(0.4,0.7),(0.3,0.7)
 51% | Tire            | /m/0h9mv   | (0.3,0.6),(0.4,0.6),(0.4,0.7),(0.3,0.7)


# Document APIs

## OCR

```
pip install google-cloud-documentai
```

In [20]:
import os

PROJECT_ID = os.environ.get("PROJECT_ID")
print(PROJECT_ID)

None


In [30]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
import os
from dotenv import load_dotenv

load_dotenv()

PROJECT_ID = os.environ.get("PROJECT_ID")
LOCATION = os.environ.get("LOCATION")  # Format is 'us' or 'eu'
PROCESSOR_ID = os.environ.get("PROCESSOR_ID")  # Create processor in Cloud Console

# The local file in your current working directory
FILE_PATH = "Demo.pdf"
# Refer to https://cloud.google.com/document-ai/docs/file-types
# for supported file types
MIME_TYPE = "application/pdf"

# Instantiates a client
docai_client = documentai.DocumentProcessorServiceClient(
    client_options=ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")
)

# The full resource name of the processor, e.g.:
# projects/project-id/locations/location/processor/processor-id
# You must create new processors in the Cloud Console first
RESOURCE_NAME = docai_client.processor_path(PROJECT_ID, LOCATION, PROCESSOR_ID)

# Read the file into memory
with open(FILE_PATH, "rb") as image:
    image_content = image.read()

# Load Binary Data into Document AI RawDocument Object
raw_document = documentai.RawDocument(content=image_content, mime_type=MIME_TYPE)

# Configure the process request
request = documentai.ProcessRequest(name=RESOURCE_NAME, raw_document=raw_document)

# Use the Document AI client to process the sample form
result = docai_client.process_document(request=request)

document_object = result.document
print("Document processing complete.")
print(f"Text: {document_object.text}")

Document processing complete.
Text: Testing pdf

